In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
from rail_coder.models import Document, DocumentCode
documents = Document.objects.all()

In [3]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Button, Layout, jslink, IntText, IntSlider, Label, Combobox, Textarea
from IPython.display import clear_output
from ipywidgets import Output
from ipywidgets import VBox, HBox
from ipywidgets import AppLayout

In [4]:
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

In [5]:
class QCoder:
    def __init__(self, doc_index=0):
        self.doc_index = doc_index
        self.doc = documents[doc_index]
        self.existing_doc_choices = self.doc.documentcode_set.all()
        self.header_out = Output(layout={'border': '1px solid black'})
        self.clause_out = Output(layout={'border': '1px solid black', 'height':"auto"})
        self.how_restricted_input = None
        self.what_restricted_input = None
        self.what_uses_input = None
        
        self.ui = self.build_ui()

    def display(self):
        display(self.ui)


    def build_header(self):
        return widgets.HTML(
            value="<h1>Clause #{} [{}]</h1>".format(self.doc.id, self.doc.source)
        )

    def build_clauses_widget(self):
        # print("build_clauses_widget({})".format(self.doc.clause_text))
        return widgets.HTML(value="<p>{}</p>".format(self.doc.clause_text))

    def build_what_restricted_ui(self):
        what_restricted_label = Label("What is the artifact being restricted?")
        
        if len(self.existing_doc_choices) > 0:
            dc = self.doc.documentcode_set.latest('date')
            placeholder = dc.artifacts_restricted
        else:
            placeholder = " -- "
        
        self.what_restricted_input = Combobox(placeholder=placeholder,
                                         options = ["licensed artifacts", "the model",
                                                    "the model or derivatives of the model",
                                                    "licensed artifacts or its derivatives",
                                                    "licensed artifcats materials",
                                                    "Artifacts or Your Derivatives, in whole or in part"],
                                         disabled=False,
                                         ensure_option=True,
                                         layout=Layout(height='auto', width='auto'),
                                        )

        return HBox([what_restricted_label, self.what_restricted_input])

    def build_what_uses_ui(self):
        what_uses_label = Label("What are the uses being restricted?")

        if len(self.existing_doc_choices) > 0:
            dc = self.doc.documentcode_set.latest('date')
            placeholder = dc.uses_restricted
        else:
            placeholder = " -- "
        
        self.what_uses_input = Textarea(
            value='',
            placeholder=placeholder,
            disabled=False
        )
        return HBox([what_uses_label, self.what_uses_input])

    def build_how_restricted_ui(self):
        how_restricted_label = Label("How are they restricted?")

        if len(self.existing_doc_choices) > 0:
            dc = self.doc.documentcode_set.latest('date')
            placeholder = dc.how_restricted
        else:
            placeholder = " -- "
        
        self.how_restricted_input = Textarea(
            value='',
            placeholder=placeholder,
            disabled=False
        )
        return HBox([how_restricted_label, self.how_restricted_input])

    def create_expanded_button(description, button_style):
        return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))
    
    def build_controls_ui(self):
        prev_b = create_expanded_button("Previous", "info")
        prev_b.on_click(self.previous)

        next_b = create_expanded_button("Next", "info")
        next_b.on_click(self.next)

        submit_b = create_expanded_button("Submit", "success")
        submit_b.on_click(self.submit)
        
        controls_ui = VBox([prev_b, next_b, submit_b], layout=Layout(height="auto", width="auto"))
        
        return controls_ui

    def submit(self, b):
        print("submit()")

        # gather the information from the question boxes
        what_restricted_value = self.what_restricted_input.value
        how_restricted_value = self.how_restricted_input.value
        what_uses_value = self.what_uses_input.value

        print("What Restricted: {}".format(what_restricted_value))
        print("How restricted: {}".format(how_restricted_value))
        print("What uses: {}".format(what_uses_value))
        
        # create a DocumentCode object from the information in the questions
        doc_code = DocumentCode(document = self.doc,
                                artifacts_restricted = what_restricted_value,
                                how_restricted = how_restricted_value,
                                uses_restricted = what_uses_value)
        doc_code.save()
    
    def next(self, b):
        print("next")
        
        if self.doc_index < len(documents):
            self.doc_index += 1
            self.doc = documents[self.doc_index]
        else:
            print("doc index > len(documents)")

        self.header_out.clear_output()
        with self.header_out:
            display(self.build_header())

        self.clause_out.clear_output()
        with self.clause_out:
            display(self.build_clauses_widget())

        self.update_question_placeholders()

    def previous(self, b):
        print("previous")
        if self.doc_index > 0:
            self.doc_index -= 1
            self.doc = documents[self.doc_index]
        else:
            print("doc index < 0")
            
        self.header_out.clear_output()
        with self.header_out:
            display(self.build_header())

        self.clause_out.clear_output()
        with self.clause_out:
            display(self.build_clauses_widget())

        self.update_question_placeholders()

    def update_question_placeholders(self):
        if len(self.existing_doc_choices) > 0:
            dc = self.doc.documentcode_set.latest('date')
            artifacts_restricted_placeholder = dc.artifacts_restricted
            what_uses_placeholder = dc.uses_restricted
            how_restricted_placeholder = dc.how_restricted
        else:
            artifacts_restricted_placeholder = " -- "
            what_uses_placeholder = " -- "
            how_restricted_placeholder = " -- "

        self.what_restricted_ui.children[1].placeholder = artifacts_restricted_placeholder
        self.how_restricted_ui.children[1].placeholder = how_restricted_placeholder
        self.what_uses_ui.children[1].placeholder = what_uses_placeholder
        
    def build_ui(self):
        header = self.header_out
        with header:
            display(self.build_header())
        
        clause_widget = self.clause_out
        with clause_widget:
            display(self.build_clauses_widget())

        self.what_restricted_ui = self.build_what_restricted_ui()
        self.what_uses_ui = self.build_what_uses_ui()
        self.how_restricted_ui = self.build_how_restricted_ui()
                
        questions_box = VBox([self.what_restricted_ui, self.what_uses_ui, self.how_restricted_ui], align_items="top", layout=Layout(height="auto", width="auto"))
        controls_ui = self.build_controls_ui()
        
        applo = AppLayout(header=header,
                      left_sidebar=None,
                      center=clause_widget,
                      # center=create_expanded_button('Center', 'warning'),
                      right_sidebar=controls_ui,
                      # footer=create_expanded_button('Footer', 'success'),
                      footer=questions_box,
                      align_items='top')
        return applo

In [9]:
qcoder = QCoder(3)
qcoder.display()

AppLayout(children=(Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', borde…